In [1]:
import gradio as gr
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import logging
import instructor

# Load environment variables
load_dotenv("config.env")

# Get API Keys from environment variables
CLIENT_ID = os.getenv("IMGUR_CLIENT_ID")
CLIENT_SECRET = os.getenv("IMGUR_CLIENT_SECRET")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

# Imgur API endpoint for image upload
IMGUR_UPLOAD_URL = "https://api.imgur.com/3/image"

# Initialize OpenAI client
client = instructor.from_openai(
    OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY,
    )
)

/home/ubuntu/Desktop/myracle/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pydantic import BaseModel
from typing import List, Optional


class TestCase(BaseModel):
    name: str
    description: str
    code: str


class TestCasesResponse(BaseModel):
    test_cases: List[TestCase]

In [3]:
def generate_description(image_url, user_description):
    """Generate a description using the OpenAI API."""
    completion = client.chat.completions.create(
        model="google/gemini-flash-8b-1.5-exp",
        response_model=TestCasesResponse,
        messages=[
            {
                "role": "system",
                "content": """
                <system_prompt>
                You are a QA automation expert tasked with creating Python test scripts for mobile applications based on screenshots. For each screenshot provided:

                1. Analyze the UI elements, features, and functionality visible.
                2. Generate a comprehensive set of test cases covering all major functionality.
                3. Create a Python script using pytest that implements these test cases.

                Your response should be a JSON object with the following structure:
                {
                "test_cases": [
                    {
                    "name": "Test case name",
                    "description": "Brief description of the test case",
                    "code": "Python code implementing the test case"
                    },
                    ...
                ]
                }

                Each test case should:

                - Use pytest as the testing framework.
                - Assume the existence of a MobileApp class that represents the app under test.
                - Include setup and teardown methods as needed.
                - Implement the test case as a separate test function.
                - Use descriptive function names prefixed with `test_`.
                - Include comments explaining the purpose of the test.
                - Use assertions to verify expected outcomes.

                Focus on user interactions, edge cases, and potential error scenarios. Ensure test cases are specific, actionable, and cover both positive and negative testing scenarios.
                </system_prompt>

                <example>
                {
                "test_cases": [
                    {
                    "name": "test_search_available_buses",
                    "description": "Verify user can search for available buses",
                    "code": "```python\ndef test_search_available_buses(self):\n    \"\"\"Verify user can search for available buses\"\"\"\n    self.app.enter_source('Mumbai')\n    self.app.enter_destination('Pune')\n    self.app.select_journey_date('Sat, 7 Sep')\n    self.app.tap_search_button()\n    assert self.app.is_bus_list_displayed(), 'Bus list should be displayed after search'\n```"
                    },
                    {
                    "name": "test_date_selection",
                    "description": "Verify date selection functionality",
                    "code": "```python\ndef test_date_selection(self):\n    \"\"\"Verify date selection functionality\"\"\"\n    self.app.tap_date_field()\n    assert self.app.is_quick_date_option_visible('TODAY'), 'TODAY option should be visible'\n    assert self.app.is_quick_date_option_visible('TOMORROW'), 'TOMORROW option should be visible'\n    \n    self.app.select_quick_date_option('TODAY')\n    today_date = self.app.get_selected_date()\n    assert today_date == self.app.get_current_date(), 'Selected date should be today'\n\n    self.app.select_quick_date_option('TOMORROW')\n    tomorrow_date = self.app.get_selected_date()\n    assert tomorrow_date == self.app.get_tomorrow_date(), 'Selected date should be tomorrow'\n```"
                    },
                    {
                    "name": "test_cab_bus_hire_navigation",
                    "description": "Verify cab/bus hire functionality",
                    "code": "```python\ndef test_cab_bus_hire_navigation(self):\n    \"\"\"Verify cab/bus hire functionality\"\"\"\n    self.app.tap_ryde_option()\n    assert self.app.is_cab_bus_hire_screen_displayed(), 'Cab/Bus hire screen should be displayed'\n```"
                    }
                ]
                }
                </example>
                """,
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_description,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": image_url},
                    },
                ],
            },
        ],
    )

    return completion.choices[0].message.content

In [4]:
img_url = "https://i.imgur.com/Ty9uaej.jpeg"

In [6]:
image_url = img_url
user_description = "Write a test case for the image"

In [7]:
completion = client.chat.completions.create(
    model="google/gemini-flash-8b-1.5-exp",
    response_model=TestCasesResponse,
    messages=[
        {
            "role": "system",
            "content": """
            <system_prompt>
            You are a QA automation expert tasked with creating Python test scripts for mobile applications based on screenshots. For each screenshot provided:

            1. Analyze the UI elements, features, and functionality visible.
            2. Generate a comprehensive set of test cases covering all major functionality.
            3. Create a Python script using pytest that implements these test cases.

            Your response should be a JSON object with the following structure:
            {
            "test_cases": [
                {
                "name": "Test case name",
                "description": "Brief description of the test case",
                "code": "Python code implementing the test case"
                },
                ...
            ]
            }

            Each test case should:

            - Use pytest as the testing framework.
            - Assume the existence of a MobileApp class that represents the app under test.
            - Include setup and teardown methods as needed.
            - Implement the test case as a separate test function.
            - Use descriptive function names prefixed with `test_`.
            - Include comments explaining the purpose of the test.
            - Use assertions to verify expected outcomes.

            Focus on user interactions, edge cases, and potential error scenarios. Ensure test cases are specific, actionable, and cover both positive and negative testing scenarios.
            </system_prompt>

            <example>
            {
            "test_cases": [
                {
                "name": "test_search_available_buses",
                "description": "Verify user can search for available buses",
                "code": "```python\ndef test_search_available_buses(self):\n    \"\"\"Verify user can search for available buses\"\"\"\n    self.app.enter_source('Mumbai')\n    self.app.enter_destination('Pune')\n    self.app.select_journey_date('Sat, 7 Sep')\n    self.app.tap_search_button()\n    assert self.app.is_bus_list_displayed(), 'Bus list should be displayed after search'\n```"
                },
                {
                "name": "test_date_selection",
                "description": "Verify date selection functionality",
                "code": "```python\ndef test_date_selection(self):\n    \"\"\"Verify date selection functionality\"\"\"\n    self.app.tap_date_field()\n    assert self.app.is_quick_date_option_visible('TODAY'), 'TODAY option should be visible'\n    assert self.app.is_quick_date_option_visible('TOMORROW'), 'TOMORROW option should be visible'\n    \n    self.app.select_quick_date_option('TODAY')\n    today_date = self.app.get_selected_date()\n    assert today_date == self.app.get_current_date(), 'Selected date should be today'\n\n    self.app.select_quick_date_option('TOMORROW')\n    tomorrow_date = self.app.get_selected_date()\n    assert tomorrow_date == self.app.get_tomorrow_date(), 'Selected date should be tomorrow'\n```"
                },
                {
                "name": "test_cab_bus_hire_navigation",
                "description": "Verify cab/bus hire functionality",
                "code": "```python\ndef test_cab_bus_hire_navigation(self):\n    \"\"\"Verify cab/bus hire functionality\"\"\"\n    self.app.tap_ryde_option()\n    assert self.app.is_cab_bus_hire_screen_displayed(), 'Cab/Bus hire screen should be displayed'\n```"
                }
            ]
            }
            </example>
            """,
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_description,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": image_url},
                },
            ],
        },
    ],
)

InstructorRetryException: RetryError[<Future at 0xf52c7e51f9d0 state=finished raised RateLimitError>]